# Extraction, transformation, loading of weather data

In [0]:
from pyspark import SparkContext
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType
from pyspark.sql import SQLContext, SparkSession
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.sql import types
import pyspark.sql.functions as F
SEED = 7

import warnings
warnings.filterwarnings('ignore')

In [0]:
file_path = "dbfs:/mnt/mids-w261"
weather_src = "weather_data"
username = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply('user')
private_files = 'dbfs:/user/' + username
project_path = "dbfs:/user/chitra.agastya@ischool.berkeley.edu/FinalProject/"
# Enable for pretty viewing of tables
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)    
    

In [0]:
# global flags
verbose = True
save = True

In [0]:
if verbose:
    display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project/"+weather_src))




path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/teamjvml1/,teamjvml1/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather-miss.parquet/,weather-miss.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2015a.parquet/,weather2015a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2016a.parquet/,weather2016a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2017a.parquet/,weather2017a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2018a.parquet/,weather2018a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2019a.parquet/,weather2019a.parquet/,0


## Load the data

In [0]:
weather = spark.read.option("header", "true").parquet(f"{file_path}/datasets_final_project/{weather_src}/*.parquet")

# not doing this next bit anymore, the join will take care of it
#weather = weather.filter(f"(date - interval {interval} {unit}) < '{start}'")

In [0]:
display()

In [0]:
if verbose:
    display(weather.limit(10))
    weather.printSchema()
weather.createOrReplaceTempView("weather")

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AW1,GA1,GA2,GA3,GA4,GE1,GF1,KA1,KA2,MA1,MD1,MW1,MW2,OC1,OD1,OD2,REM,EQD,AW2,AX4,GD1,AW5,GN1,AJ1,AW3,MK1,KA4,GG3,AN1,RH1,AU5,HL1,OB1,AT8,AW7,AZ1,CH1,RH3,GK1,IB1,AX1,CT1,AK1,CN2,OE1,MW5,AO1,KA3,AA3,CR1,CF2,KB2,GM1,AT5,AY2,MW6,MG1,AH6,AU2,GD2,AW4,MF1,AA1,AH2,AH3,OE3,AT6,AL2,AL3,AX5,IB2,AI3,CV3,WA1,GH1,KF1,CU2,CT3,SA1,AU1,KD2,AI5,GO1,GD3,CG3,AI1,AL1,AW6,MW4,AX6,CV1,ME1,KC2,CN1,UA1,GD5,UG2,AT3,AT4,GJ1,MV1,GA5,CT2,CG2,ED1,AE1,CO1,KE1,KB1,AI4,MW3,KG2,AA2,AX2,AY1,RH2,OE2,CU3,MH1,AM1,AU4,GA6,KG1,AU3,AT7,KD1,GL1,IA1,GG2,OD3,UG1,CB1,AI6,CI1,CV2,AZ2,AD1,AH1,WD1,AA4,KC1,IA2,CF3,AI2,AT1,GD4,AX3,AH4,KB3,CU1,CN4,AT2,CG1,CF1,GG1,MV2,CW1,GG4,AB1,AH5,CN3
7650099999,2016-01-01T00:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-12,99999,V020,"190,1,N,0015,1","99999,9,9,N","007000,1,9,9","+0101,1","+0090,1","10255,1",,"99,9,+02250,1,99,9",,,,"9,AGL ,+99999,+99999","08,99,1,99,9,99,9,02250,1,99,9,99,9",,,"99999,9,10216,1","8,1,004,1,+999,9","61,1",,,"3,99,0026,1,999",,SYN09807650 04857 81903 10101 20090 30216 40255 58004 69901 761// 333 4/000 69907 90710 91105 555 69905=,,,,,,,"9999,1,9,999999,9,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"03,0000,2,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"06,0000,2,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T00:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-15,99999,V020,"190,1,N,0015,1","22000,1,9,N","009000,1,9,9","+0100,1","+0090,1","99999,9","61,1",,,,,,"00,99,1,99,9,99,9,99999,9,99,9,99,9",,,"10250,1,99999,9",,,,,,,MET057METAR LFML 010000Z AUTO 19003KT 9000 -RA NSC 10/09 Q1025=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T00:30:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-15,99999,V020,"250,1,N,0010,1","99999,9,9,N","008000,1,9,9","+9999,9","+9999,9","99999,9",,,,,,,,,,"10250,1,99999,9",,,,,,,MET056METAR LFML 010030Z AUTO 25002KT 8000 ///TCU 10/09 Q1025=,Q019 2ATOD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T01:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-12,99999,V020,"999,9,C,0000,1","99999,9,9,N","004900,1,9,9","+0099,1","+0094,1","10251,1",,"99,9,+02250,1,99,9",,,,"9,AGL ,+99999,+99999","08,99,1,99,9,99,9,02250,1,99,9,99,9",,,"99999,9,10212,1","8,1,006,1,+999,9","10,1",,,"3,99,0015,1,999",,SYN07607650 24849 80000 10099 20094 30212 40251 58006 710// 333 69925 90710 91103=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"01,0002,3,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T01:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-15,99999,V020,"999,9,C,0000,1","22000,1,9,N","007000,1,9,9","+0100,1","+0090,1","99999,9",,,,,,,"00,99,1,99,9,99,9,99999,9,99,9,99,9",,,"10250,1,99999,9",,,,,,,MET053METAR LFML 010100Z AUTO 00000KT 7000 NSC 10/09 Q1025=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T01:30:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-15,99999,V020,"999,9,C,0000,1","22000,1,9,N","009000,1,9,9","+0100,1","+0090,1","99999,9",,,,,,,"00,99,1,99,9,99,9,99999,9,99,9,99,9",,,"10250,1,99999,9",,,,,,,MET053METAR LFML 010130Z AUTO 00000KT 9000 NSC 10/09 Q1025=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T02:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-12,99999,V020,"999,9,C,0000,1","99999,9,9,N","011000,1,9,9","+0100,1","+0088,1","10254,1",,"99,9,+01750,1,99,9",,,,"9,AGL ,+99999,+99999","08,99,1,99,9,99,9,01750,1,99,9,99,9",,,"99

root
-- STATION: string (nullable = true)
-- DATE: timestamp (nullable = true)
-- SOURCE: short (nullable = true)
-- LATITUDE: double (nullable = true)
-- LONGITUDE: double (nullable = true)
-- ELEVATION: double (nullable = true)
-- NAME: string (nullable = true)
-- REPORT_TYPE: string (nullable = true)
-- CALL_SIGN: string (nullable = true)
-- QUALITY_CONTROL: string (nullable = true)
-- WND: string (nullable = true)
-- CIG: string (nullable = true)
-- VIS: string (nullable = true)
-- TMP: string (nullable = true)
-- DEW: string (nullable = true)
-- SLP: string (nullable = true)
-- AW1: string (nullable = true)
-- GA1: string (nullable = true)
-- GA2: string (nullable = true)
-- GA3: string (nullable = true)
-- GA4: string (nullable = true)
-- GE1: string (nullable = true)
-- GF1: string (nullable = true)
-- KA1: string (nullable = true)
-- KA2: string (nullable = true)
-- MA1: string (nullable = true)
-- MD1: string (nullable = true)
-- MW1: string (nullable = true)
-- MW2: string (nullable = true)
-- OC1: string (nullable = true)
-- OD1: string (nullable = true)
-- OD2: string (nullable = true)
-- REM: string (nullable = true)
-- EQD: string (nullable = true)
-- AW2: string (nullable = true)
-- AX4: string (nullable = true)
-- GD1: string (nullable = true)
-- AW5: string (nullable = true)
-- GN1: string (nullable = true)
-- AJ1: string (nullable = true)
-- AW3: string (nullable = true)
-- MK1: string (nullable = true)
-- KA4: string (nullable = true)
-- GG3: string (nullable = true)
-- AN1: string (nullable = true)
-- RH1: string (nullable = true)
-- AU5: string (nullable = true)
-- HL1: string (nullable = true)
-- OB1: string (nullable = true)
-- AT8: string (nullable = true)
-- AW7: string (nullable = true)
-- AZ1: string (nullable = true)
-- CH1: string (nullable = true)
-- RH3: string (nullable = true)
-- GK1: string (nullable = true)
-- IB1: string (nullable = true)
-- AX1: string (nullable = true)
-- CT1: string (nullable = true)
-- AK1: string (nullable = true)
-- CN2: string (nullable = true)
-- OE1: string (nullable = true)
-- MW5: string (nullable = true)
-- AO1: string (nullable = true)
-- KA3: string (nullable = true)
-- AA3: string (nullable = true)
-- CR1: string (nullable = true)
-- CF2: string (nullable = true)
-- KB2: string (nullable = true)
-- GM1: string (nullable = true)
-- AT5: string (nullable = true)
-- AY2: string (nullable = true)
-- MW6: string (nullable = true)
-- MG1: string (nullable = true)
-- AH6: string (nullable = true)
-- AU2: string (nullable = true)
-- GD2: string (nullable = true)
-- AW4: string (nullable = true)
-- MF1: string (nullable = true)
-- AA1: string (nullable = true)
-- AH2: string (nullable = true)
-- AH3: string (nullable = true)
-- OE3: string (nullable = true)
-- AT6: string (nullable = true)
-- AL2: string (nullable = true)
-- AL3: string (nullable = true)
-- AX5: string (nullable = true)
-- IB2: string (nullable = true)
-- AI3: string (nullable = true)
-- CV3: string (nullable = true)
-- WA1: string (nullable = true)
-- GH1: string (nullable = true)
-- KF1: string (nullable = true)
-- CU2: string (nullable = true)
-- CT3: string (nullable = true)
-- SA1: string (nullable = true)
-- AU1: string (nullable = true)
-- KD2: string (nullable = true)
-- AI5: string (nullable = true)
-- GO1: string (nullable = true)
-- GD3: string (nullable = true)
-- CG3: string (nullable = true)
-- AI1: string (nullable = true)
-- AL1: string (nullable = true)
-- AW6: string (nullable = true)
-- MW4: string (nullable = true)
-- AX6: string (nullable = true)
-- CV1: string (nullable = true)
-- ME1: string (nullable = true)
-- KC2: string (nullable = true)
-- CN1: string (nullable = true)
-- UA1: string (nullable = true)
-- GD5: string (nullable = true)
-- UG2: string (nullable = true)
-- AT3: string (nullable = true)
-- AT4: string (nullable = true)
-- GJ1: string (nullable = true)
-- MV1: string (nullable = true)
-- GA5: string (nullable = true)
-- CT2: string (nullable = true)
-- CG2: string

## Parsing

In [0]:
# unpack combination weather info into arrays

data = spark.sql("""
    select 
        call_sign,
        date,
        split(wnd, ',') as wnd, 
        split(cig, ',') as cig,
        split(vis, ',') as vis,
        split(tmp, ',') as tmp,
        split(dew, ',') as dew,
        split(slp, ',') as slp
    from
        weather
    where 
        (substring(call_sign,1,1)=='K' or substring(call_sign,1,1)=='P' or substring(call_sign,1,1)=='T')
        and substring(call_sign,5,1)==' '
        and (report_type = 'FM-15' or report_type == 'FM-16')
""")
data.createOrReplaceTempView("weather_arrays")
#if verbose:
    #display(data.sample(False, 0.0001))
  #data.count()

Explanation of parsed mandatory weather info:

According to https://www1.ncdc.noaa.gov/pub/data/noaa/isd-format-document.pdf:
Common to many fields are "quality codes". These are on the following scale:
0 = Passed gross limits check
1 = Passed all quality control checks
2 = Suspect
3 = Erroneous
4 = Passed gross limits check, data originate from an NCEI data source
5 = Passed all quality control checks, data originate from an NCEI data source
6 = Suspect, data originate from an NCEI data source
7 = Erroneous, data originate from an NCEI data source
9 = Passed gross limits check if element is present

Encoded fields:
station is a catalog identifier. We will ignore it.
date is the timestamp we use as one of two relational keys to connect us to flight data.
source describes the type and frequency of station and report, we will ignore this.
latitude and longitutude are geographic coordinates we could use to connect to airports in flights, but we have an easier method available.
name is the friendly name of the station, we will ignore this.
report_type tells us what kind of weather report we are dealing with. We will focus exclusively on METAR routine (FM-15) and SPECI special (FM-16) aviation weather reports, and ignore all others.
call_sign is very important to us - it the the standardized short name for weather stations in the US, consisting of a prefix letter ("K" for airport stations), a code (the airport code for the report types we care about), and a space (or longer code, this is a 5-char-field). Since we are only dealing with domestic flights, we can use this together with date to directly link flights and weather. This call_sign is consistently available for weather report_types FM-15 and FM-16. We will prove this with an outer join with flights, and a count of reports for each flight.

WND wind speed made of 5 different components, separated by commas: "020,5,N,0103,5": Direction angle (clockwise degree angle with true North, here 20 deg), direction quality code (5 meaning "good"), the type of measurement (here "N": "Normal"), speed (in m/s with a scaling factor of 10), speed quality code (5 indicating "good").

CIG ceiling made of 4 components, separated by commas: "22000,5,9,N": ceiling height in meters (22 km here), a quality code for it (5 here indicating "good"), a code for how the height was determined (9 indicating "missing" here, curious for an airport station), and a flag about whether 'Ceiling and Visibility Okay' (CAVOK) condition has been reported, "no" in this case.

VIS visibility made up of 4 components, separated by commas: "016093,5,N,5": Visibility distance in meters, a quality code for it, visibility variability (here "Not variable"), and a quality code for it.

TMP air temperature comprised of 2 components separated by commas: "-0178,5": temperature in degrees Celsius with a scaling factor of 10, and an expanded quality code that can flag special circumstances with letter values. Our example means -17.8 degrees Celsius, with good quality.
DEW the dew point temperature, "The temperature to which a given parcel of air must be cooled at constant pressure and water vapor content in order for saturation to occur". In degrees celsius with a scaling factor of 10, and a quality code. Example: "-0233,5" meaning -23.3 degrees Celsius with good quality.
SLP the pressure adjusted to sea-level, 2 values separated by commas: "10155,5": First, the pressure in hectopascal with a scaling factor of 10 (so 1015.5 hectopascal here) and then another uality code.
other columns: There are many more columns in this report, mostly optional, and we will ignore them.

In [0]:
# parse selected data points from the arrays

data = spark.sql ("""
    select
        call_sign, 
        date,
        
        element_at(wnd, 1) as wind_angle,
        element_at(wnd, 2) as wind_angle_quality,
        element_at(wnd, 4)/10 as wind_speed,
        element_at(wnd, 5) as wind_speed_quality,
        
        element_at(cig, 1) as ceiling,
        element_at(cig, 2) as ceiling_quality,
        
        element_at(vis, 1) as visibility,
        element_at(vis, 2) as visibility_quality,
        
        element_at(tmp, 1)/10 as temperature, 
        element_at(tmp, 2) as temperature_quality,
        
        element_at(dew, 1)/10 as dewpoint, 
        element_at(dew, 2) as dewpoint_quality,
        
        element_at(slp, 1)/10 as pressure,
        element_at(slp, 2) as pressure_quality
    from
        weather_arrays
""")

data.createOrReplaceTempView("weather_parsed")
# if verbose:
#     display(data.sample(False, 0.0001))
# data.count()

In [0]:
# filter, leave only highest quality data

hqd = "(0,1,4,5,9)" # 1,5 == passed all quality checks. 0,4,9 == passed gross limit checks.

data = spark.sql(f"""
    select
        call_sign, 
        date,      
        if(wind_angle_quality in {hqd}, wind_angle, NULL) as wind_angle,
        if(wind_speed_quality in {hqd}, wind_speed, NULL) as wind_speed,     
        if(ceiling_quality in {hqd}, ceiling, NULL) as ceiling,
        if(visibility_quality in {hqd},visibility,NULL) as visibility, 
        if(temperature_quality in {hqd}, temperature, NULL) as temperature,
        if(dewpoint_quality in {hqd}, dewpoint, NULL) as dewpoint,
        if(pressure_quality in {hqd}, pressure, NULL) as pressure
    from 
        weather_parsed
""")
data.createOrReplaceTempView("weather_quality")
# if verbose:
#     display(data.sample(False, 0.001))
# data.count()

In [0]:
data.printSchema()

root
-- call_sign: string (nullable = true)
-- date: timestamp (nullable = true)
-- wind_angle: string (nullable = true)
-- wind_speed: double (nullable = true)
-- ceiling: string (nullable = true)
-- visibility: string (nullable = true)
-- temperature: double (nullable = true)
-- dewpoint: double (nullable = true)
-- pressure: double (nullable = true)

We have a little work to do because the call_sign join doesn't work for 27 airports. We will ignore 2 of them - PSE and OGS - because they are minor and have no call_sign-tagged weather associated with them that we could find. The remaining 25 will now have their call_signs replaced, and the result is the new "airport" column for the join.

In [0]:
# File location and type
file_location = "/FileStore/tables/team_30_weather_replacements.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df.createOrReplaceTempView("call_signs")

In [0]:
data = spark.sql(f"""
    select
        coalesce(call_signs.airport, substring(weather_quality.call_sign,2,3)) as airport,
        date,      
        wind_angle,
        wind_speed,     
        ceiling,
        visibility, 
        temperature,
        dewpoint,
        pressure
    from 
        weather_quality left outer join call_signs
    on
        weather_quality.call_sign == call_signs.weather   
""")
data.createOrReplaceTempView("weather")


In [0]:
save = True
if save:
    data.write.mode("overwrite").parquet(project_path+f"weather_all.parquet")

In [0]:
display(dbutils.fs.ls("dbfs:/user/chitra.agastya@ischool.berkeley.edu/FinalProject/"))

path,name,size
dbfs:/user/chitra.agastya@ischool.berkeley.edu/FinalProject/Chitra_feature_data/,Chitra_feature_data/,0
dbfs:/user/chitra.agastya@ischool.berkeley.edu/FinalProject/airline_singleday.parquet/,airline_singleday.parquet/,0
dbfs:/user/chitra.agastya@ischool.berkeley.edu/FinalProject/airlines_3m_features_ext.parquet/,airlines_3m_features_ext.parquet/,0
dbfs:/user/chitra.agastya@ischool.berkeley.edu/FinalProject/airlines_3m_full_features.parquet/,airlines_3m_full_features.parquet/,0
dbfs:/user/chitra.agastya@ischool.berkeley.edu/FinalProject/airlines_weather_data/,airlines_weather_data/,0
dbfs:/user/chitra.agastya@ischool.berkeley.edu/FinalProject/airport-timezones.csv,airport-timezones.csv,439779
dbfs:/user/chitra.agastya@ischool.berkeley.edu/FinalProject/airport_edges/,airport_edges/,0
dbfs:/user/chitra.agastya@ischool.berkeley.edu/FinalProject/airport_edges_1_year/,airport_edges_1_year/,0
dbfs:/user/chitra.agastya@ischool.berkeley.edu/FinalProject/airport_edges_3_month/,airport_edges_3_month/,0
dbfs:/user/chitra.agastya@ischool.berkeley.edu/FinalProject/airport_edges_4_year/,airport_edges_4_year/,0


In [0]:
display(spark.read.option("header", "true").parquet(project_path+"weather_all.parquet/*.parquet").limit(10))
print("done with weather")

airport,date,wind_angle,wind_speed,ceiling,visibility,temperature,dewpoint,pressure
DTA,2017-01-01T06:55:00.000+0000,180,2.6,00030,000000,null,null,9999.9
DTA,2017-01-01T07:15:00.000+0000,170,2.1,00030,000000,-15.0,-16.0,9999.9
DTA,2017-01-01T07:35:00.000+0000,999,0.0,00030,000000,-15.0,-16.0,9999.9
DTA,2017-01-01T07:55:00.000+0000,999,0.0,00030,000000,null,null,9999.9
DTA,2017-01-01T08:15:00.000+0000,999,0.0,00030,000000,-15.0,-17.0,9999.9
DTA,2017-01-01T08:35:00.000+0000,120,2.1,00030,000000,-15.0,-16.0,9999.9
DTA,2017-01-01T08:55:00.000+0000,070,2.1,00030,000000,null,null,9999.9
DTA,2017-01-01T09:15:00.000+0000,070,2.1,00030,000402,-14.0,-15.0,9999.9
DTA,2017-01-01T09:35:00.000+0000,999,0.0,00030,000402,-14.0,-15.0,9999.9
DTA,2017-01-01T09:55:00.000+0000,070,2.1,00030,000000,null,null,9999.9


done with weather